In [17]:
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler

def preprocess_states_dataset(file_path, output_csv_path):
    # Charger les données
    df = pd.read_csv(file_path, delimiter='$')

    # Suppression des colonnes inutiles
    columns_to_drop = ['Unnamed: 0', 'mun_code']
    for col in columns_to_drop:
        if col in df.columns:
            df.drop(columns=[col], inplace=True)

    # Renommer les colonnes
    columns_to_rename = {
        'lat': 'Latitude',
        'lon': 'Longitude',
        'mun_name': 'state_name'
    }
    df.rename(columns=columns_to_rename, inplace=True)

    # Ajouter des colonnes factices pour correspondre à l'ensemble d'entraînement
    df['Speed_limit'] = 0  # Valeur factice
    df['Time'] = 0         # Valeur factice

    # Nettoyage des noms de colonnes
    df.columns = df.columns.str.strip()

    # Validation des coordonnées
    if 'Longitude' in df.columns and 'Latitude' in df.columns:
        df = df[(df['Longitude'] >= -180) & (df['Longitude'] <= 180)]
        df = df[(df['Latitude'] >= -90) & (df['Latitude'] <= 90)]

 # Charger le scaler et le modèle KMeans
    scaler = joblib.load('./models/scaler2.pkl')
    kmeans = joblib.load('./models/kmeans_model.pkl')


    # Normaliser toutes les colonnes nécessaires
    df[['Longitude', 'Latitude',  'Time' ,'Speed_limit']] = scaler.transform(
        df[['Longitude', 'Latitude', 'Time', 'Speed_limit']]
    )
    df['Longitude']= df['Longitude'].round(4)
    df['Latitude']= df['Latitude'].round(4)

    # Ajouter les clusters basés sur les coordonnées
    df['Location_Cluster'] = kmeans.predict(df[['Longitude', 'Latitude']])

    # Supprimer les colonnes factices après le clustering
    df.drop(columns=['Speed_limit', 'Time'], inplace=True)

    # Sauvegarder le dataset nettoyé et clusterisé
    df.to_csv(output_csv_path, index=False, sep=';')
    print(f"Données nettoyées et clusterisées sauvegardées dans : {output_csv_path}")

    return df

# Appel de la fonction
processed_df =preprocess_states_dataset("./part1/stn_UK.csv", "result_roud4_scaler2.csv")

processed_df

Données nettoyées et clusterisées sauvegardées dans : result_roud4_scaler2.csv


,osm_name,st_name,state_name,Longitude,Latitude,Location_Cluster
0,"""77 Steps""",77 steps,Rainhill,-2.7579,53.4189,3
1,Allendale Avenue,allendale avenue,Rainhill,-2.7597,53.4164,3
2,Alness Drive,alness drive,Rainhill,-2.7565,53.4105,3
3,Amanda Road,amanda road,Rainhill,-2.7727,53.4235,3
4,Anderson Close,anderson close,Rainhill,-2.7546,53.4068,3
...,...,...,...,...,...,...
918074,Woolwich Foot Tunnel,woolwich foot tunnel,Woolwich Riverside,0.0623,51.4965,3
918075,Worms Head path,worms head path,Gower,-4.3269,51.5651,3
918076,Worsdell Street,worsdell street,Sleekburn,-1.5104,55.1354,3
918077,Wye Bridge,wye bridge,Thornwell,-2.6608,51.6177,3


In [19]:
filtered_df = processed_df[processed_df['Location_Cluster'] == 1] # Display the filtered DataFrame 
filtered_df

,osm_name,st_name,state_name,Longitude,Latitude,Location_Cluster
